In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = r'/Users/sebreawilmore/Instacart Basket Analysis'

# 1.

In [3]:
orders_products_merge = pd.read_pickle(r'/Users/sebreawilmore/Instacart Basket Analysis/Data/orders_products_combined.pkl')

# 2. 

In [4]:
#Finding average number of orders in each department:
orders_products_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


# 3.

All of the departments are listed in the output. Only a few departments were listed in the sample 

# 4.

In [7]:
#Creating loyalty flag
orders_products_merge['max_order'] = orders_products_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [8]:
orders_products_merge.head(5)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,...,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_order,spending_flag,average_order,user_frequency,frequency_flag
0,2539329.0,1.0,1.0,2.0,8.0,NaN,196.0,1.0,0.0,195.0,...,Mid-range product,Regularly busy,Average orders,10.0,New customer,6.367797,Low spender,6.367797,20.5,Non frequent customer
1,2398795.0,1.0,2.0,3.0,7.0,15.0,196.0,1.0,1.0,195.0,...,Mid-range product,Least busy days,Average orders,10.0,New customer,6.367797,Low spender,6.367797,20.5,Non frequent customer
2,473747.0,1.0,3.0,3.0,12.0,21.0,196.0,1.0,1.0,195.0,...,Mid-range product,Least busy days,Most orders,10.0,New customer,6.367797,Low spender,6.367797,20.5,Non frequent customer
3,2254736.0,1.0,4.0,4.0,7.0,29.0,196.0,1.0,1.0,195.0,...,Mid-range product,Least busy days,Average orders,10.0,New customer,6.367797,Low spender,6.367797,20.5,Non frequent customer
4,431534.0,1.0,5.0,4.0,15.0,28.0,196.0,1.0,1.0,195.0,...,Mid-range product,Least busy days,Most orders,10.0,New customer,6.367797,Low spender,6.367797,20.5,Non frequent customer


In [9]:
#Assigning loyalty labels to users
orders_products_merge.loc[orders_products_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
orders_products_merge.loc[(orders_products_merge['max_order'] <= 40) & (orders_products_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
orders_products_merge.loc[orders_products_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
orders_products_merge[['user_id', 'loyalty_flag', 'order_number']].head(5)

,user_id,loyalty_flag,order_number
0,1.0,New customer,1.0
1,1.0,New customer,2.0
2,1.0,New customer,3.0
3,1.0,New customer,4.0
4,1.0,New customer,5.0


# 5.

In [11]:
orders_products_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

The mean for loyal customers is lower than new & regular customers. The minimum and maximum is the same for each group.

# 6.

In [12]:
#Creating spending flag
orders_products_merge['average_order'] = orders_products_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [13]:
#Assigning spending flag to users
orders_products_merge.loc[orders_products_merge['average_order'] < 10, 'spending_flag'] = 'Low spender'
orders_products_merge.loc[orders_products_merge['average_order'] >= 10, 'spending_flag'] = 'High spender'

In [14]:
orders_products_merge[['user_id', 'average_order', 'spending_flag']].head(15)

,user_id,average_order,spending_flag
0,1.0,6.367797,Low spender
1,1.0,6.367797,Low spender
2,1.0,6.367797,Low spender
3,1.0,6.367797,Low spender
4,1.0,6.367797,Low spender
5,1.0,6.367797,Low spender
6,1.0,6.367797,Low spender
7,1.0,6.367797,Low spender
8,1.0,6.367797,Low spender
9,1.0,6.367797,Low spender


In [19]:
#Creating frequency flag
orders_products_merge['user_frequency'] = orders_products_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [21]:
#Assigning frequency flag
orders_products_merge.loc[orders_products_merge['user_frequency'] > 20, 'frequency_flag'] = 'Non frequent customer'
orders_products_merge.loc[(orders_products_merge['user_frequency'] >10) & (orders_products_merge['user_frequency'] <= 20), 'frequency_flag'] = 'Regular customer'
orders_products_merge.loc[orders_products_merge['user_frequency'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [22]:
orders_products_merge[['user_id', 'user_frequency', 'frequency_flag']].head(15)

,user_id,user_frequency,frequency_flag
0,1.0,20.5,Non frequent customer
1,1.0,20.5,Non frequent customer
2,1.0,20.5,Non frequent customer
3,1.0,20.5,Non frequent customer
4,1.0,20.5,Non frequent customer
5,1.0,20.5,Non frequent customer
6,1.0,20.5,Non frequent customer
7,1.0,20.5,Non frequent customer
8,1.0,20.5,Non frequent customer
9,1.0,20.5,Non frequent customer


In [25]:
orders_products_merge.head(2)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,...,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,mean_order,spending_flag,average_order,user_frequency,frequency_flag
0,2539329.0,1.0,1.0,2.0,8.0,NaN,196.0,1.0,0.0,195.0,...,Mid-range product,Regularly busy,Average orders,10.0,New customer,6.367797,Low spender,6.367797,20.5,Non frequent customer
1,2398795.0,1.0,2.0,3.0,7.0,15.0,196.0,1.0,1.0,195.0,...,Mid-range product,Least busy days,Average orders,10.0,New customer,6.367797,Low spender,6.367797,20.5,Non frequent customer


# Exporting Updated Dataframe

In [23]:
orders_products_merge.to_pickle(os.path.join(path, 'Data', 'orders_products_combined.pkl'))